In [3]:
from pyspark.sql import SparkSession

# The rest of your code will now work
spark = SparkSession.builder \
    .appName("JupyterSparkTest") \
    .master("local[*]") \
    .getOrCreate()

print("Spark session successfully created in Jupyter!")
print(spark.version)

Spark session successfully created in Jupyter!
4.0.0


In [7]:
import os
from pyspark.sql.functions import col

# --- 1. Define the directory where all your JSON files are ---
unzipped_dir = "local_cricsheet_data/t20_json"

# --- 2. Get a list of all filenames and select a small sample ---
# We'll filter for .json files just to be safe
all_json_files = [f for f in os.listdir(unzipped_dir) if f.endswith('.json')]
sample_files = all_json_files[:5] # Let's test with just the first 5 files

# --- 3. Create the full, absolute paths for the sample files ---
sample_file_paths = [os.path.join(unzipped_dir, f) for f in sample_files]

print("Testing with the following files:")
for path in sample_file_paths:
    print(f" - {path}")

# --- 4. Read ONLY the sample files into a Spark DataFrame ---
# spark.read.json() can accept a list of file paths directly.
df_sample = spark.read.option("multiLine", "true").json(sample_file_paths)

# --- 5. Verify the read and schema ---
print("\nSchema of the sample DataFrame:")
df_sample.printSchema()
print(f"\nSuccessfully loaded {df_sample.count()} files into the sample DataFrame.")

Testing with the following files:
 - local_cricsheet_data/t20_json\1001349.json
 - local_cricsheet_data/t20_json\1001351.json
 - local_cricsheet_data/t20_json\1001353.json
 - local_cricsheet_data/t20_json\1004729.json
 - local_cricsheet_data/t20_json\1007655.json

Schema of the sample DataFrame:
root
 |-- info: struct (nullable = true)
 |    |-- balls_per_over: long (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- dates: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- event: struct (nullable = true)
 |    |    |-- match_number: long (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |-- gender: string (nullable = true)
 |    |-- match_type: string (nullable = true)
 |    |-- match_type_number: long (nullable = true)
 |    |-- missing: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- officials: struct (nullable = true)
 |    |    |-- match_referees: array (nullable = true)

In [15]:
from pyspark.sql.functions import col

# Use the .select() transformation to flatten the nested 'info' struct
# We use dot notation to access nested fields and .alias() to rename them.
final_df_sample = df_sample.select(
    col("info.city").alias("city"),
    col("info.dates")[0].alias("match_date"),
    col("info.event.name").alias("event_name"),
    col("info.gender").alias("gender"),
    col("info.match_type").alias("match_type_detail"),
    col("info.outcome.winner").alias("winner"),
    col("info.outcome.by.wickets").alias("win_by_wickets"),
    col("info.outcome.by.runs").alias("win_by_runs"),
    col("info.overs").alias("overs"),
    col("info.player_of_match")[0].alias("player_of_match"),
    col("info.teams")[0].alias("team_1"),
    col("info.teams")[1].alias("team_2"),
    col("info.toss.decision").alias("toss_decision"),
    col("info.toss.winner").alias("toss_winner"),
    col("info.venue").alias("venue")
)

# The .show() action executes the transformation and displays the result
print("Transformed and flattened data from the sample files:")
final_df_sample.show()

Transformed and flattened data from the sample files:
+-----------+----------+--------------------+------+-----------------+---------+--------------+-----------+-----+---------------+---------+---------+-------------+-----------+--------------------+
|       city|match_date|          event_name|gender|match_type_detail|   winner|win_by_wickets|win_by_runs|overs|player_of_match|   team_1|   team_2|toss_decision|toss_winner|               venue|
+-----------+----------+--------------------+------+-----------------+---------+--------------+-----------+-----+---------------+---------+---------+-------------+-----------+--------------------+
|       NULL|2016-06-18|India in Zimbabwe...|  male|              T20| Zimbabwe|          NULL|          2|   20|   E Chigumbura| Zimbabwe|    India|        field|      India|  Harare Sports Club|
|   Victoria|2017-02-19|Sri Lanka in Aust...|  male|              T20|Sri Lanka|             2|       NULL|   20|  DAS Gunaratne|Australia|Sri Lanka|        f

In [17]:
from pyspark.sql.functions import year as get_year, lit

# Add columns that are good for partitioning.
# 1. Add a 'year' column by extracting it from the 'match_date'.
df_with_year = final_df_sample.withColumn("year", get_year(col("match_date")))

# 2. Add a 'match_type' column with a fixed value.
partitioned_df = df_with_year.withColumn("match_type", lit("t20"))


# Define a local output path.
local_output_path = "local_cricsheet_data/processed_parquet"

print(f"Writing partitioned data to: '{local_output_path}'")
partitioned_df.printSchema()


# Write the DataFrame to Parquet format.
# This is an action that triggers the previous transformations.
partitioned_df.write.partitionBy("match_type", "year").mode("overwrite").parquet(local_output_path)

print("\nWrite complete!")

Writing partitioned data to: 'local_cricsheet_data/processed_parquet'
root
 |-- city: string (nullable = true)
 |-- match_date: string (nullable = true)
 |-- event_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- match_type_detail: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- win_by_wickets: long (nullable = true)
 |-- win_by_runs: long (nullable = true)
 |-- overs: long (nullable = true)
 |-- player_of_match: string (nullable = true)
 |-- team_1: string (nullable = true)
 |-- team_2: string (nullable = true)
 |-- toss_decision: string (nullable = true)
 |-- toss_winner: string (nullable = true)
 |-- venue: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- match_type: string (nullable = false)


Write complete!
